In [1]:
import tensorflow as tf
print(f"Tensorflow version : {tf.__version__}")

Tensorflow version : 2.2.0-rc1


In [14]:
from tensorflow.python.ops import sparse_ops
import tensorflow.keras.backend as K
import pandas as pd
# import matplotlib.pyplot as plt
import os
# assert tf.test.is_gpu_available()
print(tf.config.list_physical_devices('GPU'))
import matplotlib.pyplot as plt
import numpy as np
from tableauColors import palette

[]


In [3]:
from ipynb.fs.full.clustering import make_clusters, extract_features, tf_serialize_example

Tensorflow version : 2.2.0-rc1


## Parser

In [4]:
feature_description = {
    'eventId': tf.io.FixedLenFeature([1], tf.int64, default_value=0),
    'image': tf.io.FixedLenFeature([3], tf.string, default_value=["",]*3)
}

feature_shape = {
    'eventId': tf.TensorShape([1,]),
    'image': tf.TensorShape([568, 568, 2])
}

def parser_fn(proto):
    serialized = tf.io.parse_single_example(proto, feature_description)
    deserialized = {k: (tf.sparse.to_dense(sparse_ops.deserialize_sparse(v, K.floatx()))
                        if k != 'eventId' else v) for k,v in serialized.items()}
    [deserialized[k].set_shape(feature_shape[k]) for k in deserialized.keys()]
    x = deserialized['image']
    x = tf.expand_dims(x, axis=0)
    x = K.pool2d(x, pool_size=(2, 2), strides=(2, 2), pool_mode='avg')
    # sum instead of avg
    x = 4.*x
    deserialized['image'] = tf.squeeze(x)
    deserialized['S_image'] = deserialized['image'][:,:,0]
    deserialized['C_image'] = deserialized['image'][:,:,1]
    
    deserialized['tseed'] = tf.constant(4., tf.float32)
    deserialized['tneighbour'] = tf.constant(2., tf.float32)
    deserialized['tcell'] = tf.constant(0., tf.float32)
    deserialized['tenergy'] = tf.constant(500., tf.float32)
    deserialized['tlocmax'] = tf.constant(250., tf.float32)
    deserialized['tnum'] = tf.constant(3, tf.int32)
    
    return deserialized

In [6]:
DATA_DIR = '/groups/hep/mojen/repositories/DREAM/Run/final_run/tauolaevts/10000'
# DATA_DIR = '/home/jupyter/DREAM'
BATCH_SIZE = 32
BUFFER_SIZE = 64

filename = [os.path.join(DATA_DIR, 'filtered.tfrecord')]
dataset = tf.data.TFRecordDataset(filename, compression_type='GZIP', buffer_size=BUFFER_SIZE)
dataset = dataset.take(10).map(parser_fn)
clusters_dataset = dataset.map(make_clusters)
features_dataset = clusters_dataset.map(extract_features)
S_serialized_features_dataset = features_dataset.map(lambda x: tf_serialize_example(x, 'S'))
C_serialized_features_dataset = features_dataset.map(lambda x: tf_serialize_example(x, 'C'))

In [15]:
# list(dataset.map(lambda x: tf.reduce_max(x['S_image'])).as_numpy_iterator())

In [10]:
# import datetime
# print(datetime.datetime.now())

2020-05-29 10:40:46.817647


In [11]:
# writer = tf.data.experimental.TFRecordWriter(os.path.join(DATA_DIR, 'S_cluster.tfrecord'))
# writer.write(S_serialized_features_dataset)

In [12]:
# import datetime
# print(datetime.datetime.now())

2020-05-29 10:40:58.715627


In [8]:
# writer = tf.data.experimental.TFRecordWriter(os.path.join(DATA_DIR, 'C_cluster.tfrecord'))
# writer.write(C_serialized_features_dataset)

In [9]:
# datetime.datetime.now()

datetime.datetime(2020, 5, 25, 15, 34, 49, 530903)

In [7]:
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames)
for raw_record in raw_dataset.take(4):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

DataLossError: corrupted record at 0

In [224]:
import ROOT
from array import array

# palette_lst = [v for k,v in palette.items()]
palette_lst = [None, None]
palette_lst[0] = [palette['orange'], palette['green'], palette['red'], 
               palette['cyan'], palette['pink'], palette['purple']]
palette_lst[1] = [palette['orange'], palette['red'], palette['green'], 
               palette['cyan'], palette['pink'], palette['purple']]
ztitle = {'S': 'Scintillating signal [GeV]', 
          'C': '#check{C}erenkov signal [GeV]'}

ratio = 1.1
offset = -568/4+0.5
htot = [None, None]

def plot_output(output):
    h = ROOT.TH2F("","", 284, -142., 142., 284, -142., 142.)
    c = ROOT.TCanvas("c1","c1", 1200, 600)
    c.Divide(2)
    ROOT.gStyle.SetOptStat(0)
    for l,X in enumerate(['S', 'C']):
        c.cd(l+1)
        ROOT.gPad.SetLeftMargin(0.15)
        ROOT.gPad.SetRightMargin(0.2)
        ROOT.gPad.SetTopMargin(0.15)
        ROOT.gPad.SetBottomMargin(0.2)
        X_image = output[X+'_image']
        indices = tf.where(X_image)
        values = tf.reshape(tf.gather_nd(X_image, indices), [-1,1])
        X_image = tf.concat([tf.cast(indices, values.dtype), values], axis=1)

        htot[l] = h.Clone()
        for i,j,x in X_image:
            htot[l].Fill(j+offset, i+offset, x)
        htot[l].GetZaxis().SetTitle(ztitle[X])
        htot[l].GetXaxis().SetTitle('X coordinate [3 mm]')
        htot[l].GetYaxis().SetTitle('Y coordinate [3 mm]')
        
        htot[l].GetXaxis().CenterTitle()
        htot[l].GetYaxis().CenterTitle()
        htot[l].GetZaxis().CenterTitle()
        
        htot[l].GetXaxis().SetTitleOffset(htot[l].GetXaxis().GetTitleOffset() * ratio)
        htot[l].GetYaxis().SetTitleOffset(htot[l].GetYaxis().GetTitleOffset() * ratio)
        htot[l].GetZaxis().SetTitleOffset(htot[l].GetZaxis().GetTitleOffset() * (ratio + 0.6))

        htot[l].GetXaxis().SetTitleSize(htot[l].GetXaxis().GetTitleSize() * (ratio + 0.2))
        htot[l].GetYaxis().SetTitleSize(htot[l].GetYaxis().GetTitleSize() * (ratio + 0.2))
        htot[l].GetZaxis().SetTitleSize(htot[l].GetZaxis().GetTitleSize() * (ratio + 0.2))
        
        htot[l].GetXaxis().SetLabelSize(htot[l].GetXaxis().GetLabelSize() * ratio)
        htot[l].GetYaxis().SetLabelSize(htot[l].GetYaxis().GetLabelSize() * ratio)
        htot[l].GetZaxis().SetLabelSize(htot[l].GetZaxis().GetLabelSize() * ratio)
        
        htot[l].GetXaxis().SetTickSize(htot[l].GetXaxis().GetTickLength() * ratio)
        htot[l].GetYaxis().SetTickSize(htot[l].GetYaxis().GetTickLength() * ratio)
        htot[l].GetZaxis().SetTickSize(htot[l].GetZaxis().GetTickLength() * ratio)
        
        htot[l].GetXaxis().SetLabelOffset(htot[l].GetXaxis().GetLabelOffset() * ratio)
        htot[l].GetYaxis().SetLabelOffset(htot[l].GetYaxis().GetLabelOffset() * ratio)
        htot[l].GetZaxis().SetLabelOffset(htot[l].GetZaxis().GetLabelOffset() * ratio)
        
#         htot[l].SetAxisRange(50., 200., 'X')
#         htot[l].SetAxisRange(50., 200., 'Y')
#         htot[l].SetAxisRange(0., 1200., 'Z')
        htot[l].SetAxisRange(-95., 95., 'X')
        htot[l].SetAxisRange(-95., 95., 'Y')
        htot[l].SetAxisRange(0., 1200., 'Z')
        
        htot[l].GetYaxis().SetNdivisions(504)
        htot[l].GetXaxis().SetNdivisions(504)
        
        htot[l].Draw("COLZ")
#         htot[l].GetXaxis().ChangeLabel(1,-1,-1,11)
#         htot[l].GetYaxis().ChangeLabel(1,-1,-1,31)

        X_cluster = output[X+'_cluster'].numpy()
        if X_cluster.size != 0:
            hk = []
            u = np.unique(X_cluster[:,0], axis=0).astype(int)
            arr = array('d',[0.5])
            for k in range(len(u)):
                for i,j,x in X_cluster[X_cluster[...,0]==u[k]][:,1:]:
                    hk.append(h.Clone())
                    hk[k].Fill(j+offset, i+offset, x)

            for k in range(len(u)):
                hk[k].SetLineColor(palette_lst[l][k].GetNumber())
                hk[k].SetLineWidth(linewidth)
                hk[k].SetContour(1, arr)
                hk[k].DrawCopy("cont3 list same")
                
    c.SaveAs("cluster"+str(output['eventId'][0].numpy())+".png")

In [225]:
for output in clusters_dataset.take(4):
    plot_output(output)

Info in <TCanvas::Print>: png file cluster0.png has been created
Info in <TCanvas::Print>: png file cluster1.png has been created
Info in <TCanvas::Print>: png file cluster3.png has been created
Info in <TCanvas::Print>: png file cluster4.png has been created


In [193]:
pd.read_csv(os.path.join(DATA_DIR, 'truth.csv'))

,eventId,PrimaryDecayMode,showerId,VecShowerPDG,IsCharged,VecShowerEnergy,VecShowerScntCoMi,VecShowerScntCoMj,VecShowerCkovCoMi,VecShowerCkovCoMj
0,0,4,0,-211,1,21408.335401,297.601513,210.178398,301.278854,211.544229
1,0,4,1,22,0,924.508020,265.292563,227.143003,267.097902,228.489510
2,0,4,2,22,0,1100.245997,194.759711,444.109299,193.218310,444.767606
3,0,4,3,22,0,1643.766190,322.681255,343.655726,322.418605,344.218605
4,0,4,4,22,0,10000.615035,270.707679,318.412099,270.388983,318.429661
5,1,2,6,-211,1,7879.859769,226.777910,443.623535,210.570149,452.304478
6,3,0,9,11,1,11217.589660,197.923195,323.988905,197.600430,323.923242
7,4,4,11,-211,1,6158.922797,124.973960,292.176013,126.284133,287.734317
8,4,4,12,22,0,952.328476,269.464007,211.364788,269.992000,211.744000
9,4,4,13,22,0,5571.902271,320.498040,295.250833,321.379549,296.188908
